In [1]:
import cv2
import mediapipe as mp
import random

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 웹캠 캡처
cap = cv2.VideoCapture(0)

# Pose 모델 초기화
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    random_x, random_y = -1, -1  # 초기화 (빨간 점은 처음에 화면에 안 나옴)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # BGR에서 RGB로 변환
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Pose 추적
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            # 왼쪽 손목의 랜드마크 위치
            left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
            h, w, _ = frame.shape
            left_wrist_x, left_wrist_y = int(left_wrist.x * w), int(left_wrist.y * h)

            # 빨간 점이 이미 화면에 있으면, 점이 손목과 가까워졌는지 확인
            if random_x == -1 and random_y == -1:  # 빨간 점이 없으면 새로 생성
                random_x = random.randint(0, w)
                random_y = random.randint(0, h)

            # 빨간 점 그리기
            cv2.circle(frame, (random_x, random_y), 10, (0, 0, 255), -1)

            # 손목과 점의 거리가 가까우면 'complete' 표시
            if abs(left_wrist_x - random_x) < 30 and abs(left_wrist_y - random_y) < 30:
                cv2.putText(frame, "Complete", (w // 2 - 100, h // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                # 'complete'가 표시된 후, 새로운 점을 생성하도록 처리
                random_x = random.randint(0, w)
                random_y = random.randint(0, h)

            # 인체 랜드마크를 파란색으로 그리기
            landmark_spec = mp_styles.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4)
            connection_spec = mp_styles.DrawingSpec(color=(255, 0, 0), thickness=2)

            # 인체 랜드마크 그리기 (파란색)
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                landmark_drawing_spec=landmark_spec,
                connection_drawing_spec=connection_spec
            )

        # 화면에 결과 출력
        cv2.imshow("Pose Estimation", frame)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


### 스쿼트 영상에 모델 적용 및 시각화 처리

In [1]:
import torch
from lstm import LSTMmodel

# 디바이스 설정
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')

# 모델 정의
squat_model = 'model/squat_lstm_train.pt'
model = LSTMmodel(3,10).to(device)

# 저장된 가중치 불러오기
model.load_state_dict(torch.load(squat_model, map_location=device, weights_only=True))
# model.eval() # 평가 모드로 전환
print(model)

LSTMmodel(
  (lstm): LSTM(3, 10, num_layers=2, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=10, out_features=3, bias=True)
    (1): Tanh()
  )
)


In [9]:
import cv2
import numpy as np
import mediapipe as mp

# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 비디오 파일 열기
cap = cv2.VideoCapture('squat_data/squat_001.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 원본 프레임 크기 저장 (결과를 맞추기 위해)
    original_frame = frame.copy()

    # Mediapipe는 RGB 이미지를 처리하므로 변환
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    # 예상 노드 좌표 (23, 24, 25, 26, 27, 28) 저장
    if results.pose_landmarks:
        input_data = []
        for node_id in [23, 24, 25, 26, 27, 28]:
            landmark = results.pose_landmarks.landmark[node_id]
            input_data.append([node_id, landmark.x, landmark.y, landmark.z])

        # 입력 데이터를 텐서로 변환
        input_tensor = torch.tensor([item[1:] for item in input_data], dtype=torch.float32).unsqueeze(0).to(device)

        # 모델 예측
        with torch.no_grad():
            predicted_output = model(input_tensor)

        # 예측된 좌표 출력 (예측된 출력이 2D 배열일 경우)
        predicted_coords = predicted_output.squeeze().cpu().detach().numpy()

        # 예측된 좌표가 2D 배열인지 확인
        if predicted_coords.ndim == 1:
            predicted_coords = predicted_coords.reshape(-1, 3)  # 1D array를 3개의 값으로 변환

        # 실제 좌표와 예측된 좌표를 비교하여 빨간 점 그리기
        for i, (input_item, predicted_item) in enumerate(zip(input_data, predicted_coords)):
            x_real, y_real, z_real = input_item[1], input_item[2], input_item[3]
            x_pred, y_pred, z_pred = predicted_item  # 예측된 값이 3개의 좌표값

            # 빨간 점을 실제 좌표에 그리기
            if abs(x_real - x_pred) > 0.05 or abs(y_real - y_pred) > 0.05:  # 차이가 큰 경우만 빨간 점
                cv2.circle(frame, (int(x_pred * frame.shape[1]), int(y_pred * frame.shape[0])), 5, (0, 0, 255), -1)
            else:
                cv2.circle(frame, (int(x_real * frame.shape[1]), int(y_real * frame.shape[0])), 5, (0, 255, 0), -1)

    # 결과 영상 표시
    cv2.imshow("Squat Pose Prediction", frame)

    # 종료 조건 (q 키)
    if cv2.waitKey(1) == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()

### 각 노드별 나타내기가 가능한지 확인

In [ ]:
import cv2
import mediapipe as mp
import torch

# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 비디오 파일 열기
cap = cv2.VideoCapture('squat_data/squat_001.mp4')

# 예상된 노드 좌표를 위한 변수 초기화
predicted_coords = None
random_coords = {23: (-1, -1), 24: (-1, -1), 25: (-1, -1), 26: (-1, -1), 27: (-1, -1), 28: (-1, -1)}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 원본 프레임 크기 저장 (결과를 맞추기 위해)
    original_frame = frame.copy()

    # Mediapipe는 RGB 이미지를 처리하므로 변환
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        input_data = []
        for node_id in [23, 24, 25, 26, 27, 28]:
            landmark = results.pose_landmarks.landmark[node_id]
            input_data.append([node_id, landmark.x, landmark.y, landmark.z])

        # 입력 데이터를 텐서로 변환
        input_tensor = torch.tensor([item[1:] for item in input_data], dtype=torch.float32).unsqueeze(0).to(device)

        # 모델 예측
        with torch.no_grad():
            predicted_output = model(input_tensor)

        # 출력 형태 확인
        # print(predicted_output.shape)  # 예측된 출력의 shape 확인
        predicted_coords = predicted_output.squeeze().cpu().detach().numpy()

        # 각 노드에 대해 빨간 점을 그리거나 사라지게 처리
        for i, (input_item, predicted_item) in enumerate(zip(input_data, predicted_coords)):
            node_id = input_item[0]
            x_real, y_real, z_real = input_item[1], input_item[2], input_item[3]
            
            # 예측된 좌표가 배열로 반환되는지 확인
            if isinstance(predicted_item, np.ndarray):
                # 예측된 3D 좌표(x, y, z)를 올바르게 할당
                x_pred, y_pred, z_pred = predicted_item[0], predicted_item[1], predicted_item[2]
            else:
                # 예측된 값이 스칼라인 경우 처리
                x_pred = predicted_item
                y_pred = predicted_item
                z_pred = predicted_item

            # 화면 좌표로 변환
            screen_x = int(x_pred * frame.shape[1])
            screen_y = int(y_pred * frame.shape[0])

            # 빨간 점이 없으면 새로 생성
            if random_coords[node_id] == (-1, -1):
                random_coords[node_id] = (screen_x, screen_y)

            # 빨간 점 그리기
            cv2.circle(frame, random_coords[node_id], 10, (0, 0, 255), -1)

            # 실제 좌표와 예측된 좌표 차이를 비교하여 점 사라짐
            if abs(x_real - x_pred) < 0.05 and abs(y_real - y_pred) < 0.05:
                # 예측된 빨간 점을 실제 좌표에 맞추어 사라지게
                random_coords[node_id] = (-1, -1)

        # 인체 랜드마크 그리기 (파란색)
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )

    # 화면에 결과 출력
    cv2.imshow("Pose Estimation", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [24]:
import cv2
import mediapipe as mp
import torch
import numpy as np

# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 비디오 파일 열기
cap = cv2.VideoCapture('squat_data/squat_001.mp4')

# 예상된 노드 좌표를 위한 변수 초기화
predicted_coords = None
random_coords = {23: (-1, -1), 24: (-1, -1), 25: (-1, -1), 26: (-1, -1), 27: (-1, -1), 28: (-1, -1)}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 원본 프레임 크기 저장 (결과를 맞추기 위해)
    original_frame = frame.copy()

    # Mediapipe는 RGB 이미지를 처리하므로 변환
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        input_data = []
        for node_id in [23, 24, 25, 26, 27, 28]:
            landmark = results.pose_landmarks.landmark[node_id]
            input_data.append([node_id, landmark.x, landmark.y, landmark.z])

        # 입력 데이터를 텐서로 변환
        input_tensor = torch.tensor([item[1:] for item in input_data], dtype=torch.float32).unsqueeze(0).to(device)

        # 모델 예측
        with torch.no_grad():
            predicted_output = model(input_tensor)

        # 예측된 좌표 처리 (3D 좌표)
        predicted_coords = predicted_output.squeeze().cpu().detach().numpy()

        # 각 노드에 대해 빨간 점을 그리거나 사라지게 처리
        for i, (input_item, predicted_item) in enumerate(zip(input_data, predicted_coords)):
            node_id = input_item[0]
            x_real, y_real, z_real = input_item[1], input_item[2], input_item[3]
            x_pred, y_pred, z_pred = predicted_item  # 예측된 좌표

            # 화면 좌표로 변환 (예상 좌표)
            screen_x_pred = int(x_pred * frame.shape[1])
            screen_y_pred = int(y_pred * frame.shape[0])

            # 빨간 점을 예측된 좌표에 생성
            cv2.circle(frame, (screen_x_pred, screen_y_pred), 10, (0, 0, 255), -1)

            # 예측된 점과 실제 점을 비교하여 동작 완료 여부 판단
            if abs(x_real - x_pred) < 0.05 and abs(y_real - y_pred) < 0.05:
                random_coords[node_id] = (-1, -1)

        # 인체 랜드마크 그리기 (파란색)
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )

    # 화면에 결과 출력
    cv2.imshow("Pose Estimation", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


TypeError: cannot unpack non-iterable numpy.float32 object

### 참고 코드 활용

In [4]:
import cv2
import mediapipe as mp
import torch
import numpy as np

# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 해상도 설정 (가로, 세로)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# 프레임 속도 설정
cap.set(cv2.CAP_PROP_FPS, 30)

# 임시 모델 (실제 모델로 교체해야 함)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_squat_1 = torch.load('model_path.pth').to(device)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # Mediapipe는 RGB 이미지를 처리하므로 변환
    frame = cv2.resize(frame, (640, 480))
    frame2 = frame.copy()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)

    # 관절 연결 및 포인트 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
        )
        mp_drawing.draw_landmarks(
            frame2,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
        )

    # 입력 데이터 준비
    input_data = []
    for node_id in [23, 24, 25, 26, 27, 28]:  # 예시로 6개 노드 사용
        landmark = results.pose_landmarks.landmark[node_id]
        input_data.append([node_id, landmark.x, landmark.y, landmark.z])

    # 입력 데이터를 텐서로 변환
    input_tensor = torch.tensor([item[1:] for item in input_data], dtype=torch.float32).unsqueeze(0).to(device)

    # 모델 추론
    with torch.no_grad():
        predicted_output = model(input_tensor)

    # 예측된 좌표 처리 (예측 좌표)
    predicted_coords = predicted_output.squeeze().cpu().detach().numpy()

    # 예측된 좌표를 실제 웹캠 화면에 표시
    for i, (input_item, predicted_item) in enumerate(zip(input_data, predicted_coords)):
        node_id = input_item[0]
        x_real, y_real, z_real = input_item[1], input_item[2], input_item[3]

        # 예측된 좌표는 모델의 출력 (x, y, z)
        # predicted_item이 3개의 값을 포함한다고 가정하고 이를 분리
        if isinstance(predicted_item, np.ndarray):  # predicted_item이 배열이라면
            x_pred, y_pred, z_pred = predicted_item  # 3D 좌표 분리
        else:  # 예측된 값이 단일 값인 경우 (비정상적인 출력 방지)
            x_pred, y_pred, z_pred = predicted_item, predicted_item, predicted_item

        # 화면 좌표로 변환
        screen_x_pred = int(x_pred * frame.shape[1])
        screen_y_pred = int(y_pred * frame.shape[0])

        # 예측된 빨간 점을 그리기
        cv2.circle(frame2, (screen_x_pred, screen_y_pred), 10, (0, 0, 255), -1)

    # 웹캠 화면과 합성
    blended_frame = cv2.addWeighted(frame2, 0.7, black_frame, 1.3, 0)

    # 결과를 화면에 표시
    cv2.imshow("Pose Estimation with Predicted Coordinates", blended_frame)

    # 종료 조건 (q 키)
    if cv2.waitKey(1) == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_6268\3651239999.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_squat_1 = torch.load('model_path.pth').to(

FileNotFoundError: [Errno 2] No such file or directory: 'model_path.pth'

In [7]:
import cv2
import mediapipe as mp
import torch
import numpy as np

# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 웹캠 열기
cap = cv2.VideoCapture(0)
# 해상도 설정 (가로, 세로)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # 가로 해상도 설정
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # 세로 해상도 설정

# 프레임 속도 설정
cap.set(cv2.CAP_PROP_FPS, 30)  # 30 FPS로 설정
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # Mediapipe는 RGB 이미지를 처리하므로 변환
    frame = cv2.resize(frame,(640,480))
    frame2 = frame.copy()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)

    # 관절 연결 및 포인트 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
        )
        mp_drawing.draw_landmarks(
            frame2, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
        )
    input_tensor = torch.tensor(black_frame[:, :, 1], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)    

    # 6개의 랜드마크 좌표 수집
    input_data = []
    for node_id in [23, 24, 25, 26, 27, 28]:  # 예시로 6개 노드 사용
        landmark = results.pose_landmarks.landmark[node_id]
        input_data.append([node_id, landmark.x, landmark.y, landmark.z])

    # 모델 추론
    with torch.no_grad():
        reconstructed_frame = model(input_tensor)

    # 입력 데이터를 텐서로 변환 (3D로 차원 추가)
    input_tensor = torch.tensor([item[1:] for item in input_data], dtype=torch.float32).unsqueeze(0).to(device)  # 차원 3D로 변환



    # 조건부 연산: 0.5보다 크면 255, 아니면 0
    reconstructed_frame = (reconstructed_frame ==1).int() * 255

    # 모델의 출력 변환
    output_frame= reconstructed_frame.squeeze().cpu().detach().numpy().astype(np.uint8)
    
    # output_frame을 3채널로 변환 (BGR)
    output_frame_color = cv2.cvtColor(output_frame, cv2.COLOR_GRAY2BGR)

    # output_frame_color의 데이터 타입을 frame과 맞춤 (uint8)
    output_frame_color = output_frame_color.astype(np.uint8)
    
    # 웹캠 화면과 합성
    output_frame_color = np.zeros((output_frame.shape[0], output_frame.shape[1], 3), dtype=np.uint8)
    output_frame_color[:, :, 1] = output_frame # 초록 채널만 활성화
    blended_frame_2 = cv2.addWeighted(frame2, 0.7, output_frame_color, 1.3, 0)
    
    # 결과를 화면에 표시
    #cv2.imshow("reality_1", blended_frame_1)
    cv2.imshow("before",blended_frame_2)
    # 종료 조건 (q 키)


    if cv2.waitKey(1) == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()

cv2.waitKey(1)

ValueError: LSTM: Expected input to be 2D or 3D, got 4D instead